# COVID-19 data analysis

## Initialize Julia Environment

In [ ]:
using Pkg
Pkg.activate(ENV["HOME"] * "/.julia/environments/" * "covid19")
if (ENV["HOME"] * "/GitHub/covid19" in LOAD_PATH) == false
    push!(LOAD_PATH, pwd());
end
ENV["PYTHON"] = ENV["CONDA_PYTHON_EXE"]
#]add CSV HTTP GitHub DataFrames PyCall PyPlot Plots Dates Statistics TimeSeries MLJ XGBoost Measurements LsqFit NCDatasets Glob JSON3 Tables Pandas FileIO CSVFiles Missings JLD2 OnlineStats CUDAnative CuArrays
#Pkg.build("PyCall")

using Plots, Statistics
plotly()

## Load COVID-19 Data

In [ ]:
import COVID19
(covid19g, covid19us) = COVID19.load_data();

In [ ]:
fips = [covid19us[i].fips for i in 1:length(covid19us)];
county = [covid19us[i].county for i in 1:length(covid19us)];
state = [covid19us[i].province_state for i in 1:length(covid19us)];

In [ ]:
#ind = findall((county .== "Anderson") .& (state .== "Texas"))

## Plot COVID-19 Data for specific state

In [ ]:
ind = findall(state .== "Arizona");
confirmed = [Float64.(covid19us[ind[i]].confirmed) for i in 1:length(ind)];
totalconfirmed = Float64.(deepcopy(confirmed[1]));
for i = 2:length(confirmed)
    totalconfirmed .= totalconfirmed .+ confirmed[i];
end
ind0 = findall(totalconfirmed .== 0);
totalconfirmed[ind0] .= NaN

tind = 1:length(covid19us[ind[1]].confirmed);
t = covid19us[ind[1]].time[tind];

pCOVID19 = plot(t, totalconfirmed, label=state[ind[1]] * " Total")
for i = 1:length(ind)
    confirmi = Float64.(deepcopy(covid19us[ind[i]].confirmed[tind]));
    ind0 = findall(confirmi .== 0);
    confirmi[ind0] .= NaN;
    plot!(covid19us[ind[i]].time[tind], confirmi, label=county[ind[i]])
end
#plot!(ylim=(0,5))
plot(pCOVID19, xrotation=20, size=(800,500), legend=:outertopright, yscale=:log10, framestyle=:box, title=state[ind[1]] * " - Confirmed COVID-19 Cases")